## Import Log

In [1]:
from pm4py.objects.log.importer.xes import importer as xes_importer
variant = xes_importer.Variants.ITERPARSE
parameters = {variant.value.Parameters.TIMESTAMP_SORT: True,variant.value.Parameters.TIMESTAMP_KEY : 'time:timestamp'}
log = xes_importer.apply('Road_Traffic_Fine_Management_Process-Copy1.xes', variant = variant,
                         parameters=parameters)

parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

### Define some functions

In [2]:
from pm4py.util import constants
from pm4py.util.pandas_utils import check_is_dataframe, check_dataframe_columns
def filter_attribute_values(log, attribute, values, how="cases", positive=True):
    
    if check_is_dataframe(log):
        check_dataframe_columns(log)
        from pm4py.algo.filtering.pandas.attributes import attributes_filter
        if how == "events":
            return attributes_filter.apply_events(log, values,
                                                  parameters={constants.PARAMETER_CONSTANT_ATTRIBUTE_KEY: attribute,
                                                              attributes_filter.Parameters.POSITIVE: positive})
        elif how == "cases":
            return attributes_filter.apply(log, values, parameters={
                constants.PARAMETER_CONSTANT_ATTRIBUTE_KEY: attribute, attributes_filter.Parameters.POSITIVE: positive})
    else:
        from pm4py.algo.filtering.log.attributes import attributes_filter
        if how == "events":
            return attributes_filter.apply_events(log, values,
                                                  parameters={constants.PARAMETER_CONSTANT_ATTRIBUTE_KEY: attribute,
                                                              attributes_filter.Parameters.POSITIVE: positive})
        else:
            return attributes_filter.apply(log, values, parameters={
                constants.PARAMETER_CONSTANT_ATTRIBUTE_KEY: attribute, attributes_filter.Parameters.POSITIVE: positive})



import pm4py
def filter_numerics(event_attribute,value,event_log = log):
    zero_values = []
    low_values = []
    medium_values = []
    high_values = []
    really_high_values = []
    attribute_values = pm4py.get_event_attribute_values(log, (event_attribute))
    #print(attribute_values)
    #totalpayment=pd.cut(df['event:totalPaymentAmount'], 
                           # bins=[0,0.00001,50,100,999,100000], 
                          # labels=['Zero','Low', 'Medium', 'High', 'Really High'],
    for k in attribute_values.keys():
        if k > 999:
            really_high_values.append(k)
        elif k > 100 and k<= 999:
            high_values.append(k)
        elif k > 50 and k <= 100:
            medium_values.append(k)
        elif k > 0.00001 and k <= 50:
            low_values.append(k)
        else:
            zero_values.append(k)
    if value == 'Really High':
        filtered_log_cases = filter_attribute_values(event_log, attribute = event_attribute,
                                                 values = really_high_values, how="cases", positive=True )
    elif value == 'High':
        filtered_log_cases_1 = filter_attribute_values(event_log, attribute = event_attribute, 
                                                     values = really_high_values, how="cases", positive=False )

        filtered_log_cases = filter_attribute_values(filtered_log_cases_1, attribute = event_attribute, 
                                                     values = high_values, how="cases", positive=True )
    elif value == 'Medium':
        filtered_log_cases_1 = filter_attribute_values(event_log, attribute = event_attribute, 
                                                     values = really_high_values + high_values, how="cases", positive=False )

        filtered_log_cases = filter_attribute_values(filtered_log_cases_1, attribute = event_attribute, 
                                                     values = medium_values, how="cases", positive=True )
    elif value == 'Low':
        filtered_log_cases_1 = filter_attribute_values(event_log, attribute = event_attribute, 
                                                     values = really_high_values + high_values + medium_values, how="cases", positive=False )

        filtered_log_cases = filter_attribute_values(filtered_log_cases_1, attribute = event_attribute, 
                                                     values = low_values, how="cases", positive=True )
    else:
        filtered_log_cases = filter_attribute_values(event_log, attribute = event_attribute, 
                                                     values = really_high_values + high_values + medium_values + low_values, how="cases", positive=False )
    return filtered_log_cases

## Get the process variants

In [4]:
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.statistics.traces.log import case_statistics
var_counts = case_statistics.get_variant_statistics(log)

In [5]:
var_counts

[{'variant': 'Create Fine,Send Fine,Insert Fine Notification,Add penalty,Send for Credit Collection',
  'count': 56482},
 {'variant': 'Create Fine,Payment', 'count': 46371},
 {'variant': 'Create Fine,Send Fine', 'count': 20385},
 {'variant': 'Create Fine,Send Fine,Insert Fine Notification,Add penalty,Payment',
  'count': 9520},
 {'variant': 'Create Fine,Send Fine,Insert Fine Notification,Add penalty,Payment,Payment',
  'count': 3736},
 {'variant': 'Create Fine,Send Fine,Insert Fine Notification,Payment,Add penalty,Payment',
  'count': 3301},
 {'variant': 'Create Fine,Send Fine,Payment', 'count': 3131},
 {'variant': 'Create Fine,Send Fine,Insert Fine Notification,Insert Date Appeal to Prefecture,Add penalty,Send Appeal to Prefecture',
  'count': 2497},
 {'variant': 'Create Fine,Send Fine,Insert Fine Notification,Add penalty,Payment,Send for Credit Collection',
  'count': 1515},
 {'variant': 'Create Fine,Send Fine,Insert Fine Notification,Payment,Add penalty,Send for Credit Collection',


## Pre- processing

### Transaction table

In [6]:
import pm4py
strings = []
numerics = []
attributes_list = pm4py.get_event_attributes(log)
for a in attributes_list:
    attribute_values = pm4py.get_event_attribute_values(log, (a))
    type_v = type(list(attribute_values.keys())[0])
    sum_attr_values = sum(attribute_values.values())
    if type_v == str :
        strings.append(a)
    elif type_v == float or type_v == int:
        if sum_attr_values >= len(log):
            numerics.append(a)
#for attr in numerics:
    #attr_values = pm4py.get_event_attribute_values(log,attr)
    #if sum(attr_values.values()) < len(log):
        #numerics.remove(attr)
print(strings)
print(numerics)

['concept:name', 'notificationType', 'org:resource', 'dismissal', 'vehicleClass', 'lastSent']
['points', 'amount', 'totalPaymentAmount', 'article']


In [7]:
from pm4py.algo.transformation.log_to_features import algorithm as log_to_features

data, feature_names = log_to_features.apply(log, parameters={"str_ev_attr": strings, 
                    "str_tr_attr": [], "num_ev_attr": numerics, "num_tr_attr": [], "str_evsucc_attr": []})
print(len(feature_names))

200


In [8]:
import pandas as pd
df = pd.DataFrame(data, columns=feature_names)
df

,event:concept:name@Add penalty,event:concept:name@Appeal to Judge,event:concept:name@Create Fine,event:concept:name@Insert Date Appeal to Prefecture,event:concept:name@Insert Fine Notification,event:concept:name@Notify Result Appeal to Offender,event:concept:name@Payment,event:concept:name@Receive Result Appeal from Prefecture,event:concept:name@Send Appeal to Prefecture,event:concept:name@Send Fine,...,event:vehicleClass@M,event:vehicleClass@R,event:lastSent@C,event:lastSent@N,event:lastSent@P,event:lastSent@UNDEFINED,event:points,event:amount,event:totalPaymentAmount,event:article
0,1,0,1,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,125.19,0.00,158
1,1,0,1,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,125.19,0.00,158
2,1,0,1,0,1,0,1,0,0,1,...,0,0,0,1,0,0,0,62.59,42.71,158
3,1,0,1,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,62.59,0.00,158
4,1,0,1,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,62.59,0.00,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150365,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,41.00,41.00,7
150366,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,41.00,41.00,7
150367,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,41.00,41.00,7
150368,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,41.00,41.00,7


#### Drop undefined string values

In [9]:
columns = []
import re
r = '[\w\d\s:]+@UNDEFINED'
for x in df.columns:
    if re.search(r,x):
        columns.append(x)

print(columns)

['event:notificationType@UNDEFINED', 'event:lastSent@UNDEFINED']


In [10]:
df.drop(columns = columns, inplace = True)

#### Identify numeric event attributes and one-hot encode them

In [11]:
for col in df.columns:
    for val in df[col].unique():
        if val != 1 and val != 0:
            print(col)
            break

event:points
event:amount
event:totalPaymentAmount
event:article


In [12]:
amount = pd.cut(df['event:amount'], 
                            bins=[0,0.00001,50,100,999,100000], 
                            labels=['Zero','Low', 'Medium', 'High', 'Really High'],
                            include_lowest = True)
totalpayment=pd.cut(df['event:totalPaymentAmount'], 
                            bins=[0,0.00001,50,100,999,100000], 
                            labels=['Zero','Low', 'Medium', 'High', 'Really High'],
                            include_lowest = True)
article = pd.cut(df['event:article'], 
                            bins=[0,0.00001,50,100,999,100000], 
                            labels=['Zero','Low', 'Medium', 'High', 'Really High'],
                            include_lowest = True)
points = pd.cut(df['event:points'],
                       bins = [0.0,0.00001,50,100,999,100000],
                       labels = ['Zero','Low', 'Medium', 'High', 'Really High'],
                        include_lowest = True)

In [13]:
df.drop(columns = ['event:article','event:totalPaymentAmount','event:points','event:amount'], inplace = True)

In [14]:
df['event:points'] = points
df['event:article'] =article
df['event:totalPaymentAmount'] = totalpayment
df['event:amount'] = amount

In [15]:
df = pd.get_dummies(data = df, columns = {'event:article','event:points','event:totalPaymentAmount','event:amount'},prefix_sep='@')

In [16]:
df.shape

(150370, 214)

#### Drop activity names

In [17]:
columns = []
import re
r = 'event:concept:name@[\w\d\s:]+'
for x in df.columns:
    if re.search(r,x):
        columns.append(x)

print(columns)

['event:concept:name@Add penalty', 'event:concept:name@Appeal to Judge', 'event:concept:name@Create Fine', 'event:concept:name@Insert Date Appeal to Prefecture', 'event:concept:name@Insert Fine Notification', 'event:concept:name@Notify Result Appeal to Offender', 'event:concept:name@Payment', 'event:concept:name@Receive Result Appeal from Prefecture', 'event:concept:name@Send Appeal to Prefecture', 'event:concept:name@Send Fine', 'event:concept:name@Send for Credit Collection']


In [18]:
df.drop(columns = columns, inplace = True)

In [19]:
df.shape

(150370, 203)

#### Drop values with support higher than 0.95

In [20]:
df.mean().sort_values(ascending = False).head(20)

event:dismissal@NIL                0.996648
event:points@Zero                  0.976405
event:vehicleClass@A               0.975673
event:article@High                 0.703837
event:totalPaymentAmount@Zero      0.536384
event:notificationType@P           0.530199
event:amount@Low                   0.482151
event:amount@Medium                0.411438
event:totalPaymentAmount@Low       0.343280
event:lastSent@P                   0.309078
event:article@Low                  0.291454
event:lastSent@N                   0.201589
event:amount@High                  0.104043
event:totalPaymentAmount@Medium    0.099029
event:org:resource@538             0.057245
event:org:resource@550             0.052770
event:org:resource@541             0.048919
event:org:resource@537             0.046093
event:org:resource@559             0.042755
event:org:resource@557             0.036151
dtype: float64

In [21]:
for x in df.columns:
    if df[x].mean() > 0.95:
        df.drop(columns = [x], inplace = True)

## Mine frequent itemsets

In [22]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(df, min_support = 0.01,max_len = 3, use_colnames = True)

In [23]:
pd.set_option('display.max_colwidth', None)

In [24]:
frequent_itemsets.sort_values(by = 'support',ascending = False)

,support,itemsets
30,0.703837,(event:article@High)
32,0.536384,(event:totalPaymentAmount@Zero)
0,0.530199,(event:notificationType@P)
26,0.482151,(event:amount@Low)
27,0.411438,(event:amount@Medium)
...,...,...
121,0.010062,"(event:amount@Medium, event:org:resource@558)"
213,0.010049,"(event:amount@Medium, event:notificationType@P, event:org:resource@558)"
214,0.010042,"(event:article@High, event:notificationType@P, event:org:resource@558)"
319,0.010042,"(event:lastSent@P, event:amount@Medium, event:org:resource@558)"


## Mine association rules

In [26]:
from mlxtend.frequent_patterns import association_rules
rules_mlxtend1 = association_rules(frequent_itemsets, metric="lift", min_threshold=1.05)

In [27]:
pd.set_option('display.max_colwidth', None)

In [28]:
rules_mlxtend1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(event:org:resource@11),(event:notificationType@P),0.016885,0.530199,0.010221,0.605356,1.141754,0.001269,1.190444
1,(event:notificationType@P),(event:org:resource@11),0.530199,0.016885,0.010221,0.019279,1.141754,0.001269,1.002441
2,(event:org:resource@29),(event:notificationType@P),0.017949,0.530199,0.013227,0.736940,1.389931,0.003711,1.785906
3,(event:notificationType@P),(event:org:resource@29),0.530199,0.017949,0.013227,0.024948,1.389931,0.003711,1.007178
4,(event:org:resource@536),(event:notificationType@P),0.032061,0.530199,0.019472,0.607343,1.145500,0.002473,1.196467
...,...,...,...,...,...,...,...,...,...
1141,"(event:points@Low, event:article@High)",(event:totalPaymentAmount@Zero),0.023588,0.536384,0.019578,0.829997,1.547395,0.006926,2.727110
1142,"(event:totalPaymentAmount@Zero, event:article@High)",(event:points@Low),0.384059,0.023595,0.019578,0.050977,2.160508,0.010516,1.028853
1143,(event:points@Low),"(event:totalPaymentAmount@Zero, event:article@High)",0.023595,0.384059,0.019578,0.829763,2.160508,0.010516,3.618142
1144,(event:totalPaymentAmount@Zero),"(event:points@Low, event:article@High)",0.536384,0.023588,0.019578,0.036501,1.547395,0.006926,1.013401


In [29]:
rules_mlxtend1.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,1146.000000,1146.000000,1146.000000,1146.000000,1146.000000,1146.000000,1146.000000
mean,0.242583,0.242583,0.041689,0.389270,1.853163,0.014818,inf
std,0.222186,0.222186,0.061299,0.338728,1.431481,0.023710,NaN
min,0.010062,0.010062,0.010035,0.014258,1.051272,0.000535,1.001420
25%,0.032061,0.032061,0.013227,0.051213,1.248767,0.003364,1.016746
50%,0.201589,0.201589,0.017750,0.343139,1.488195,0.006189,1.129142
75%,0.411438,0.411438,0.032733,0.722559,1.886085,0.013042,1.721207
max,0.703837,0.703837,0.399714,1.000000,17.082078,0.157762,inf


## Extract the rules

In [30]:
def antecedents_list(dataframe):
    ass_rules_ant = []
    for i in range(len(dataframe)):
        if len(dataframe.loc[i,'antecedents']) == 1:
            ant = re.findall(r"frozenset\({'(event:\S+@[\w\s#]+)",str(dataframe.loc[i,'antecedents']))
            ass_rules_ant.append(ant)
        elif len(dataframe.loc[i,'antecedents']) == 2:
            antecedents = str(dataframe.loc[i,'antecedents']).split(',')
        #print(antecedents)
            ant = [re.sub(r"frozenset\({'(event:\S+@[\w\s#]+)'",r"\1",antecedents[0]), (re.sub(r"'(event:\S+@[\w\s#]+)'}\)", r"\1", antecedents[1])).strip()]
            ass_rules_ant.append(ant)
        else:
            antecedents = str(dataframe.loc[i,'antecedents']).split(',')
            ant = [re.sub(r"frozenset\({'(event:\S+@[\w\s#]+)'",r"\1",antecedents[0]), (re.sub(r"'(event:\S+@[\w\s#]+)'}\)", r"\1", antecedents[1])).strip(), (re.sub(r"'(event:\S+@[\w\s#]+)'}\)", r"\1", antecedents[2])).strip()]
            ass_rules_ant.append(ant)
        

    return ass_rules_ant

In [31]:
len(antecedents_list(rules_mlxtend1))

1146

In [32]:
def consequents_list(dataframe):
    ass_rules_cons = []
    for i in range(len(dataframe)):
        if len(dataframe.loc[i,'consequents']) == 1:
            cons = re.findall(r"frozenset\({'(event:\S+@[\w\s\#]+)",str(dataframe.loc[i,'consequents']))
            ass_rules_cons.append(cons)
        elif len(dataframe.loc[i,'consequents']) == 2:
            consequents = str(dataframe.loc[i,'consequents']).split(',')
            cons = [re.sub(r"frozenset\({'(event:\S+@[\w\s\#]+)'",r"\1",consequents[0]), (re.sub(r"'(event:\S+@[\w\s#]+)'}\)", r"\1", consequents[1])).strip()]
            ass_rules_cons.append(cons)
        else:
            consequents = str(dataframe.loc[i,'consequents']).split(',')
            cons = [re.sub(r"frozenset\({'(event:\S+@[\w\s\#]+)'",r"\1",consequents[0]), (re.sub(r"'(event:\S+@[\w\s#]+)'}\)", r"\1", consequents[1])).strip(), (re.sub(r"'(event:\S+@[\w\s#]+)'}\)", r"\1", consequents[2])).strip()]
            ass_rules_cons.append(cons)
        
        
    return ass_rules_cons

In [33]:
len(consequents_list(rules_mlxtend1))

1146

### Keep only unique combinations

In [34]:
ass_rules = zip(antecedents_list(rules_mlxtend1),consequents_list(rules_mlxtend1))
ass_rules_better = []
ass_rules_even_better = []
list_ass_rules = list(ass_rules)
#print(list_ass_rules)
for x in list_ass_rules:
    ass_rules_better.append([v for lst in x for v in lst])
#print(ass_rules_better)
#for v in ass_rules_better:
for x in ass_rules_better:
    ass_rules_even_better.append(tuple(sorted(x)))
ass_rules_even_better = list(set(ass_rules_even_better))
print(len(ass_rules_even_better))

282


In [35]:
ass_rules_even_better

[('event:amount@Low', 'event:org:resource@558'),
 ('event:amount@Low', 'event:org:resource@537'),
 ('event:amount@Medium', 'event:article@High'),
 ('event:amount@Medium', 'event:lastSent@N', 'event:org:resource@536'),
 ('event:lastSent@N',
  'event:notificationType@P',
  'event:totalPaymentAmount@Zero'),
 ('event:amount@Medium',
  'event:org:resource@559',
  'event:totalPaymentAmount@Zero'),
 ('event:article@High', 'event:notificationType@P', 'event:org:resource@541'),
 ('event:amount@Medium',
  'event:org:resource@550',
  'event:totalPaymentAmount@Zero'),
 ('event:lastSent@P',
  'event:org:resource@559',
  'event:totalPaymentAmount@Zero'),
 ('event:amount@Medium', 'event:notificationType@P', 'event:org:resource@557'),
 ('event:org:resource@558', 'event:totalPaymentAmount@Low'),
 ('event:amount@Medium', 'event:notificationType@P', 'event:org:resource@550'),
 ('event:amount@High', 'event:lastSent@N', 'event:notificationType@P'),
 ('event:amount@Medium', 'event:article@Low', 'event:lastS

### Prepare to store each association rule and the retained traces that correspond to each process variant

In [37]:
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.statistics.traces.log import case_statistics
var_counts = case_statistics.get_variant_statistics(log)
#print(var_counts[:10])
for x in var_counts:
    x['association_rule'] = []
    x['count_of_traces'] = []
print(var_counts[:10])

[{'variant': 'Create Fine,Send Fine,Insert Fine Notification,Add penalty,Send for Credit Collection', 'count': 56482, 'association_rule': [], 'count_of_traces': []}, {'variant': 'Create Fine,Payment', 'count': 46371, 'association_rule': [], 'count_of_traces': []}, {'variant': 'Create Fine,Send Fine', 'count': 20385, 'association_rule': [], 'count_of_traces': []}, {'variant': 'Create Fine,Send Fine,Insert Fine Notification,Add penalty,Payment', 'count': 9520, 'association_rule': [], 'count_of_traces': []}, {'variant': 'Create Fine,Send Fine,Insert Fine Notification,Add penalty,Payment,Payment', 'count': 3736, 'association_rule': [], 'count_of_traces': []}, {'variant': 'Create Fine,Send Fine,Insert Fine Notification,Payment,Add penalty,Payment', 'count': 3301, 'association_rule': [], 'count_of_traces': []}, {'variant': 'Create Fine,Send Fine,Payment', 'count': 3131, 'association_rule': [], 'count_of_traces': []}, {'variant': 'Create Fine,Send Fine,Insert Fine Notification,Insert Date App

### Filter the event log for each association rule and find the count of the traces corresponding to each process variant. NOTE: this takes more than 30 minutes to run, since the log is filtered for 282 association rules. 

In [38]:
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.statistics.traces.log import case_statistics
from pm4py.algo.filtering.log.attributes import attributes_filter
rules_babe = []
traces = []
for rule in ass_rules_even_better:
    if len(rule) == 2:
        rule_to_append = rule
        #regex1 = re.findall(r"event:(\S+)@([\w\s]+)",rule[0])
        regex1 = [p for tpl in re.findall(r"event:(\S+)@([\w\s#]+)",rule[0]) for p in tpl]
        if regex1[0] in ['amount','points','article','totalPaymentAmount']:
            tracefilter_log_pos_rule = filter_numerics(regex1[0],regex1[1],log)
        else:
            tracefilter_log_pos_rule= attributes_filter.apply(log, [regex1[1]],
            parameters={attributes_filter.Parameters.ATTRIBUTE_KEY: regex1[0], attributes_filter.Parameters.POSITIVE: True})
        #regex2 = re.findall(r"event:(\S+)@([\w\s]+)",rule[1])
        regex2 = [p for tpl in re.findall(r"event:(\S+)@([\w\s#]+)",rule[1]) for p in tpl]
        if regex2[0] in ['amount','points','article','totalPaymentAmount']:
              tracefilter_log_pos2_rule = filter_numerics(regex2[0],regex2[1],tracefilter_log_pos_rule)
        else:
            tracefilter_log_pos2_rule= attributes_filter.apply(tracefilter_log_pos_rule, [regex2[1]],
            parameters={attributes_filter.Parameters.ATTRIBUTE_KEY: regex2[0], attributes_filter.Parameters.POSITIVE: True})
            
        var_counts_rule = case_statistics.get_variant_statistics(tracefilter_log_pos2_rule)
        rules_babe.append(rule_to_append)
        traces.append(len(tracefilter_log_pos2_rule))
        print(rule_to_append)
        print(len(tracefilter_log_pos2_rule))
        print('-------------------------------------')
            
        
    
    elif len(rule) == 3:
        rule_to_append = rule
        regex1 = [p for tpl in re.findall(r"event:(\S+)@([\w\s#]+)",rule[0]) for p in tpl]
        if regex1[0] in ['amount','points','article','totalPaymentAmount']:
            tracefilter_log_pos_rule = filter_numerics(regex1[0],regex1[1],log)
        else:
            tracefilter_log_pos_rule= attributes_filter.apply(log, [regex1[1]],
            parameters={attributes_filter.Parameters.ATTRIBUTE_KEY: regex1[0], attributes_filter.Parameters.POSITIVE: True})
        regex2 = [p for tpl in re.findall(r"event:(\S+)@([\w\s#]+)",rule[1]) for p in tpl]
        if regex2[0] in ['amount','points','article','totalPaymentAmount']:
            tracefilter_log_pos2_rule = filter_numerics(regex2[0],regex2[1],tracefilter_log_pos_rule)
        else:
            tracefilter_log_pos2_rule= attributes_filter.apply(tracefilter_log_pos_rule, [regex2[1]],
            parameters={attributes_filter.Parameters.ATTRIBUTE_KEY: regex2[0], attributes_filter.Parameters.POSITIVE: True})
        regex3 = [p for tpl in re.findall(r"event:(\S+)@([\w\s#]+)",rule[2]) for p in tpl]
        if regex3[0] in ['amount','points','article','totalPaymentAmount']:
            tracefilter_log_pos3_rule = filter_numerics(regex3[0],regex3[1],tracefilter_log_pos2_rule)
        else:
            tracefilter_log_pos3_rule= attributes_filter.apply(tracefilter_log_pos2_rule, [regex3[1]],
            parameters={attributes_filter.Parameters.ATTRIBUTE_KEY: regex3[0], attributes_filter.Parameters.POSITIVE: True})
        var_counts_rule = case_statistics.get_variant_statistics(tracefilter_log_pos3_rule)
        rules_babe.append(rule_to_append)
        traces.append(len(tracefilter_log_pos3_rule))
        print(rule_to_append)
        print(len(tracefilter_log_pos3_rule))
        print('-------------------------------------')
        
        
    for x in var_counts:
        for y in var_counts_rule:
            if x['variant'] == y['variant']:
                x['association_rule'].append(rule_to_append)
                x['count_of_traces'].append(y['count'])
        if rule_to_append not in x['association_rule']:
            x['association_rule'].append('-')
            x['count_of_traces'].append('0')

C:\Users\Sava\AppData\Local\Temp/ipykernel_10512/1273641710.py:5: DeprecatedWarning: check_is_dataframe is deprecated as of 2.2.8 and will be removed in 3.0.0. use check_is_pandas_dataframe instead
  if check_is_dataframe(log):


('event:amount@Low', 'event:org:resource@558')
2256
-------------------------------------
('event:amount@Low', 'event:org:resource@537')
3918
-------------------------------------
('event:amount@Medium', 'event:article@High')
47846
-------------------------------------
('event:amount@Medium', 'event:lastSent@N', 'event:org:resource@536')
1682
-------------------------------------
('event:lastSent@N', 'event:notificationType@P', 'event:totalPaymentAmount@Zero')
24658
-------------------------------------
('event:amount@Medium', 'event:org:resource@559', 'event:totalPaymentAmount@Zero')
1810
-------------------------------------
('event:article@High', 'event:notificationType@P', 'event:org:resource@541')
3116
-------------------------------------
('event:amount@Medium', 'event:org:resource@550', 'event:totalPaymentAmount@Zero')
2593
-------------------------------------
('event:lastSent@P', 'event:org:resource@559', 'event:totalPaymentAmount@Zero')
2117
----------------------------------

('event:amount@Medium', 'event:article@Low', 'event:notificationType@P')
13911
-------------------------------------
('event:amount@Low', 'event:org:resource@538', 'event:totalPaymentAmount@Zero')
2129
-------------------------------------
('event:amount@Medium', 'event:lastSent@P', 'event:org:resource@559')
2486
-------------------------------------
('event:lastSent@P', 'event:org:resource@559')
2920
-------------------------------------
('event:amount@Medium', 'event:lastSent@N', 'event:org:resource@550')
1796
-------------------------------------
('event:article@Low', 'event:lastSent@P', 'event:totalPaymentAmount@Zero')
9620
-------------------------------------
('event:lastSent@N', 'event:org:resource@536')
2150
-------------------------------------
('event:dismissal@#', 'event:totalPaymentAmount@Zero')
1978
-------------------------------------
('event:article@High', 'event:org:resource@537', 'event:totalPaymentAmount@Zero')
2936
-------------------------------------
('event:lastS

('event:article@High', 'event:org:resource@559', 'event:totalPaymentAmount@Low')
2077
-------------------------------------
('event:article@High', 'event:notificationType@P', 'event:totalPaymentAmount@High')
2389
-------------------------------------
('event:amount@High', 'event:notificationType@P', 'event:totalPaymentAmount@Zero')
11130
-------------------------------------
('event:amount@Low', 'event:article@Low', 'event:totalPaymentAmount@Low')
16206
-------------------------------------
('event:lastSent@N', 'event:org:resource@550')
2327
-------------------------------------
('event:amount@Low', 'event:lastSent@P', 'event:notificationType@P')
5630
-------------------------------------
('event:amount@Medium', 'event:org:resource@538')
3923
-------------------------------------
('event:amount@Medium', 'event:totalPaymentAmount@Zero')
43834
-------------------------------------
('event:article@High', 'event:notificationType@P', 'event:totalPaymentAmount@Zero')
43040
------------------

('event:amount@Medium', 'event:org:resource@541')
3324
-------------------------------------
('event:dismissal@#', 'event:lastSent@P', 'event:notificationType@P')
1560
-------------------------------------
('event:article@High', 'event:lastSent@N', 'event:totalPaymentAmount@Zero')
18014
-------------------------------------
('event:amount@Low', 'event:org:resource@559')
3934
-------------------------------------
('event:lastSent@N', 'event:notificationType@P', 'event:org:resource@537')
1638
-------------------------------------
('event:amount@Medium', 'event:notificationType@P', 'event:org:resource@536')
2246
-------------------------------------
('event:amount@Low', 'event:org:resource@557', 'event:totalPaymentAmount@Low')
2081
-------------------------------------
('event:amount@Low', 'event:org:resource@537', 'event:totalPaymentAmount@Low')
2315
-------------------------------------
('event:article@High', 'event:org:resource@548')
1576
-------------------------------------
('event:a

### Create a data frame

In [59]:
df1 = pd.DataFrame(var_counts)
split_df = pd.DataFrame(df1['count_of_traces'].tolist(),columns = ['AR{}'.format(c) for c in range(1,len(ass_rules_even_better)+1)])
df1 = pd.concat([df1[['variant','count']], split_df], axis=1)
df1

,variant,count,AR1,AR2,AR3,AR4,AR5,AR6,AR7,AR8,...,AR273,AR274,AR275,AR276,AR277,AR278,AR279,AR280,AR281,AR282
0,"Create Fine,Send Fine,Insert Fine Notification,Add penalty,Send for Credit Collection",56482,125,539,30277,1423,23992,1706,2146,2482,...,1754,1875,4017,1706,2060,16479,4566,4630,1243,30520
1,"Create Fine,Payment",46371,1394,2084,3039,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
2,"Create Fine,Send Fine",20385,642,947,1852,0,0,1,0,5,...,1,0,0,0,0,0,0,0,744,0
3,"Create Fine,Send Fine,Insert Fine Notification,Add penalty,Payment",9520,37,69,5159,33,0,0,417,0,...,118,0,0,388,0,4375,842,854,0,0
4,"Create Fine,Send Fine,Insert Fine Notification,Add penalty,Payment,Payment",3736,11,35,1957,63,0,0,150,0,...,91,0,0,130,0,1336,285,287,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,"Create Fine,Insert Date Appeal to Prefecture,Send Fine,Insert Fine Notification,Send Appeal to Prefecture,Add penalty,Payment",1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
227,"Create Fine,Insert Date Appeal to Prefecture,Send Appeal to Prefecture,Send Fine,Payment",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
228,"Create Fine,Insert Date Appeal to Prefecture,Send Appeal to Prefecture,Send Fine,Insert Fine Notification,Payment,Add penalty,Payment",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
229,"Create Fine,Insert Date Appeal to Prefecture,Send Appeal to Prefecture,Send Fine,Insert Fine Notification,Add penalty,Payment",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### find total count

In [60]:
total_count = []
total_count = ['total count of traces',150370]
for x in df1.columns[2:]:
    c = 0
    for i in range(0,df1.shape[0]):
        c += int(df1.loc[i,x])
    total_count.append(c)
print(total_count)
new_row = dict(zip(df1.columns, total_count))
df1
df1 = df1.append(new_row, ignore_index = True)

['total count of traces', 150370, 2256, 3918, 47846, 1682, 24658, 1810, 3116, 2593, 2117, 1982, 1580, 3409, 6121, 8364, 5401, 3072, 1625, 2613, 3093, 1809, 3380, 2688, 4922, 23610, 2869, 2150, 3935, 1790, 7949, 6776, 3198, 9356, 2107, 2977, 56525, 1807, 1520, 2205, 1970, 3107, 1676, 1514, 3402, 6484, 2075, 1558, 42601, 2700, 3184, 2994, 1511, 7668, 1965, 13107, 14317, 2291, 1537, 3899, 3396, 2356, 2616, 2808, 1569, 16641, 11122, 10226, 1678, 3756, 21259, 1584, 3245, 1875, 3245, 5047, 13911, 2129, 2486, 2920, 1796, 9620, 2150, 1978, 2936, 3184, 2251, 2998, 1510, 2738, 3112, 5208, 6237, 17427, 46476, 2568, 21302, 14253, 2364, 21973, 1792, 33844, 1989, 2413, 1980, 2782, 1529, 9356, 2888, 1897, 29066, 1990, 2364, 5253, 1560, 2861, 1526, 2945, 1565, 2514, 6136, 2075, 1789, 6772, 2514, 41911, 2370, 2570, 6248, 3197, 3152, 5192, 4543, 2616, 4808, 7667, 6568, 2669, 1733, 1743, 60105, 2438, 3547, 1510, 2803, 1717, 1688, 30422, 2944, 4676, 30245, 2920, 2077, 2389, 11130, 16206, 2327, 5630, 3923,

In [61]:
len(total_count)

284

In [62]:
df1

,variant,count,AR1,AR2,AR3,AR4,AR5,AR6,AR7,AR8,...,AR273,AR274,AR275,AR276,AR277,AR278,AR279,AR280,AR281,AR282
0,"Create Fine,Send Fine,Insert Fine Notification,Add penalty,Send for Credit Collection",56482,125,539,30277,1423,23992,1706,2146,2482,...,1754,1875,4017,1706,2060,16479,4566,4630,1243,30520
1,"Create Fine,Payment",46371,1394,2084,3039,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
2,"Create Fine,Send Fine",20385,642,947,1852,0,0,1,0,5,...,1,0,0,0,0,0,0,0,744,0
3,"Create Fine,Send Fine,Insert Fine Notification,Add penalty,Payment",9520,37,69,5159,33,0,0,417,0,...,118,0,0,388,0,4375,842,854,0,0
4,"Create Fine,Send Fine,Insert Fine Notification,Add penalty,Payment,Payment",3736,11,35,1957,63,0,0,150,0,...,91,0,0,130,0,1336,285,287,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,"Create Fine,Insert Date Appeal to Prefecture,Send Appeal to Prefecture,Send Fine,Payment",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
228,"Create Fine,Insert Date Appeal to Prefecture,Send Appeal to Prefecture,Send Fine,Insert Fine Notification,Payment,Add penalty,Payment",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
229,"Create Fine,Insert Date Appeal to Prefecture,Send Appeal to Prefecture,Send Fine,Insert Fine Notification,Add penalty,Payment",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,"Create Fine,Appeal to Judge,Send Fine,Payment",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
mapping = dict(zip(df1.columns[2:],ass_rules_even_better))
mapping

{'AR1': ('event:amount@Low', 'event:org:resource@558'),
 'AR2': ('event:amount@Low', 'event:org:resource@537'),
 'AR3': ('event:amount@Medium', 'event:article@High'),
 'AR4': ('event:amount@Medium', 'event:lastSent@N', 'event:org:resource@536'),
 'AR5': ('event:lastSent@N',
  'event:notificationType@P',
  'event:totalPaymentAmount@Zero'),
 'AR6': ('event:amount@Medium',
  'event:org:resource@559',
  'event:totalPaymentAmount@Zero'),
 'AR7': ('event:article@High',
  'event:notificationType@P',
  'event:org:resource@541'),
 'AR8': ('event:amount@Medium',
  'event:org:resource@550',
  'event:totalPaymentAmount@Zero'),
 'AR9': ('event:lastSent@P',
  'event:org:resource@559',
  'event:totalPaymentAmount@Zero'),
 'AR10': ('event:amount@Medium',
  'event:notificationType@P',
  'event:org:resource@557'),
 'AR11': ('event:org:resource@558', 'event:totalPaymentAmount@Low'),
 'AR12': ('event:amount@Medium',
  'event:notificationType@P',
  'event:org:resource@550'),
 'AR13': ('event:amount@High', 

In [64]:
pr_var_ass_rules = []
for i in range(0,df1.shape[0]-1):
    for x in df1.columns[2:]:
         if int(df1.loc[i,x])/df1.loc[231,x] > 0.8:
            pr_var_ass_rules.append([(df1.loc[i,'variant']),df1.loc[i,'count'],mapping[x],
                                     df1.loc[231,x], df1.loc[i,x],df1.loc[i,x]/df1.loc[231,x]])
print(len(pr_var_ass_rules))
prvassr_df = pd.DataFrame(pr_var_ass_rules, columns = ['variant','count','association_rule','total_number_of_traces', 'number_traces_process_variant','ratio'])

90


In [65]:
pd.set_option('display.max_colwidth', None)

In [66]:
prvassr_df.set_index(['variant','count'],inplace = True)
prvassr_df


association_rule  \
variant                                                                                                               count                                                                                 
Create Fine,Send Fine,Insert Fine Notification,Add penalty,Send for Credit Collection                                 56482               (event:amount@Medium, event:lastSent@N, event:org:resource@536)   
                                                                                                                      56482   (event:lastSent@N, event:notificationType@P, event:totalPaymentAmount@Zero)   
                                                                                                                      56482  (event:amount@Medium, event:org:resource@559, event:totalPaymentAmount@Zero)   
                                                                                                                      56482  (event:amount@Medium, event:org:resource@550, event:totalPaymentAmount@Zero)   
                                                                                                                      56482     (event:lastSent@P, event:org:resource@559, event:totalPaymentAmount@Zero)   
...                                                                                                                                                                                                   ...   
Create Fine,Send Fine,Insert Fine Notification,Insert Date Appeal to Prefecture,Add penalty,Send Appeal to Prefecture 2497                             (event:dismissal@#, event:totalPaymentAmount@Zero)   
                                                                                                                      2497                                          (event:dismissal@#, event:lastSent@P)   
                                                                                                                      2497                (event:dismissal@#, event:lastSent@P, event:notificationType@P)   
                                                                                                                      2497                                  (event:dismissal@#, event:notificationType@P)   
                                                                                                                      2497   (event:dismissal@#, event:notificationType@P, event:totalPaymentAmount@Zero)   

                                                                                                                             total_number_of_traces  \
variant                                                                                                               count                           
Create Fine,Send Fine,Insert Fine Notification,Add penalty,Send for Credit Collection                                 56482                    1682   
                                                                                                                      56482                   24658   
                                                                                                                      56482                    1810   
                                                                                                                      56482                    2593   
                                                                                                                      56482                    2117   
...                                                                                                                                             ...   
Create Fine,Send Fine,Insert Fine Notification,Insert Date Appeal to Prefecture,Add penalty,Send Appeal to Prefecture 2497                     1978   
                                                                                                                      2497                     1560   
         

In [67]:
prvassr_df[prvassr_df.index == ('Create Fine,Send Fine,Insert Fine Notification,Insert Date Appeal to Prefecture,Add penalty,Send Appeal to Prefecture',2497)]

association_rule  \
variant                                                                                                               count                                                                                 
Create Fine,Send Fine,Insert Fine Notification,Insert Date Appeal to Prefecture,Add penalty,Send Appeal to Prefecture 2497           (event:dismissal@#, event:lastSent@P, event:totalPaymentAmount@Zero)   
                                                                                                                      2497                             (event:dismissal@#, event:totalPaymentAmount@Zero)   
                                                                                                                      2497                                          (event:dismissal@#, event:lastSent@P)   
                                                                                                                      2497                (event:dismissal@#, event:lastSent@P, event:notificationType@P)   
                                                                                                                      2497                                  (event:dismissal@#, event:notificationType@P)   
                                                                                                                      2497   (event:dismissal@#, event:notificationType@P, event:totalPaymentAmount@Zero)   

                                                                                                                             total_number_of_traces  \
variant                                                                                                               count                           
Create Fine,Send Fine,Insert Fine Notification,Insert Date Appeal to Prefecture,Add penalty,Send Appeal to Prefecture 2497                     1558   
                                                                                                                      2497                     1978   
                                                                                                                      2497                     1560   
                                                                                                                      2497                     1560   
                                                                                                                      2497                     1898   
                                                                                                                      2497                     1896   

                                                                                                                             number_traces_process_variant  \
variant                                                                                                               count                                  
Create Fine,Send Fine,Insert Fine Notification,Insert Date Appeal to Prefecture,Add penalty,Send Appeal to Prefecture 2497                            1353   
                                                                                                                      2497                            1621   
                                                                                                                      2497                            1353   
                                                                                                                      2497                            1353   
                                                                                                                      2497                            1616   
                                                                                                                      2497                            1616   

                                                                      